### Prerrequisites

In [13]:
# !pip install -U awscli

In [1]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

### Upload the data

In [2]:
import boto3
import numpy as np
import pandas as pd
import os
import sagemaker

os.makedirs("./data", exist_ok=True)

s3_client = boto3.client("s3")
s3_client.download_file(
    f"sagemaker-sample-files", "datasets/tabular/iris/iris.data", "./data/iris.csv"
)

df_iris = pd.read_csv("./data/iris.csv", header=None)
df_iris[4] = df_iris[4].map({"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2})
# iris = df_iris[[4, 0, 1, 2, 3]].to_numpy()
# np.savetxt("./data/iris.csv", iris, delimiter=",", fmt="%1.1f, %1.3f, %1.3f, %1.3f, %1.3f")

### Split into train and test

In [3]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_iris, test_size=0.1, random_state=42) 
val_df, test_df = train_test_split(val_df, test_size=0.5, random_state=42)
print('split train: {}, val: {}, test: {} '.format(train_df.shape[0], val_df.shape[0], test_df.shape[0]))

split train: 135, val: 7, test: 8 


### Save data in local

In [4]:
train = train_df[[4, 0, 1, 2, 3]].to_numpy()
val = val_df[[4, 0, 1, 2, 3]].to_numpy()
test =  test_df[[4, 0, 1, 2, 3]].to_numpy()

np.savetxt("./data/train.csv", train, delimiter=",", fmt="%1.1f, %1.3f, %1.3f, %1.3f, %1.3f")
np.savetxt("./data/validation.csv", val, delimiter=",", fmt="%1.1f, %1.3f, %1.3f, %1.3f, %1.3f")
np.savetxt("./data/test.csv", test, delimiter=",", fmt="%1.1f, %1.3f, %1.3f, %1.3f, %1.3f")

### Upload to S3

In [6]:
# S3 prefix
prefix = "Scikit-iris"

bucket = sagemaker_session.default_bucket()

s3_train_uri = sagemaker_session.upload_data('./data/train.csv', bucket, prefix + '/data/training')
s3_val_uri = sagemaker_session.upload_data('./data/validation.csv', bucket, prefix + '/data/validation')
s3_test_uri = sagemaker_session.upload_data('./data/test.csv', bucket, prefix + '/data/test')
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

### Create SageMaker Scikit Estimator for Model 1

In [13]:
DATA_PREFIX = 'Sikit_learn'
_job = "tree-{}".format('model')
full_output_prefix = f"{DATA_PREFIX}/model_artifacts"
s3_output_path = f"s3://{bucket}/{full_output_prefix}"
s3_output_path

's3://sagemaker-us-east-1-678287862391/Sikit_learn/model_artifacts'

In [14]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
script_path = "sicikit_learn_iris.py"

sklearn = SKLearn(
    entry_point=script_path,
    framework_version=FRAMEWORK_VERSION,
    instance_type="ml.m4.xlarge",
    role=role,
    output_path=s3_output_path,
    base_job_name=_job,
    sagemaker_session=sagemaker_session,
    hyperparameters={"max_leaf_nodes": 30},
)

### Train SKLearn Estimator on train data

In [15]:
sklearn.fit({"train": s3_train_uri, 'validation': s3_val_uri})

2022-04-16 19:16:08 Starting - Starting the training job...
2022-04-16 19:16:36 Starting - Preparing the instances for trainingProfilerReport-1650136568: InProgress
.........
2022-04-16 19:17:51 Downloading - Downloading input data...
2022-04-16 19:18:31 Training - Downloading the training image......
2022-04-16 19:19:31 Training - Training image download completed. Training in progress..2022-04-16 19:19:32,655 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-04-16 19:19:32,658 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-04-16 19:19:32,669 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-04-16 19:19:33,050 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-04-16 19:19:33,071 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-04-16 19:19:33,085 sagemaker-training-toolkit INFO     No GPUs detected 

### Create SageMaker Scikit Estimator for Model 2

In [19]:
DATA_PREFIX = 'Sikit_learn_1'
_job = "randfor-{}".format('model')
full_output_prefix = f"{DATA_PREFIX}/model_artifacts"
s3_output_path = f"s3://{bucket}/{full_output_prefix}"
s3_output_path

's3://sagemaker-us-east-1-678287862391/Sikit_learn_1/model_artifacts'

In [20]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
script_path = "sicikit_rand_class.py"

sklearn_1 = SKLearn(
    entry_point=script_path,
    framework_version=FRAMEWORK_VERSION,
    instance_type="ml.m4.xlarge",
    role=role,
    output_path=s3_output_path,
    base_job_name=_job,
    sagemaker_session=sagemaker_session,)

In [21]:
sklearn_1.fit({"train": s3_train_uri, 'validation': s3_val_uri})

2022-04-16 19:26:52 Starting - Starting the training job...
2022-04-16 19:27:19 Starting - Preparing the instances for trainingProfilerReport-1650137212: InProgress
.........
2022-04-16 19:28:47 Downloading - Downloading input data...
2022-04-16 19:29:23 Training - Downloading the training image........2022-04-16 19:30:35,487 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-04-16 19:30:35,490 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-04-16 19:30:35,502 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-04-16 19:30:35,984 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-04-16 19:30:36,002 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-04-16 19:30:36,020 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-04-16 19:30:36,031 sagemaker-training-toolkit INFO  

### Without registering to ECS

In [ ]:
#Creating endpoint and deploying model
import time
sk_endpoint_name = 'sk-iris-model'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
sk_predictor = sklearn.deploy(initial_instance_count=1,
                              instance_type='ml.m5.4xlarge',
                              endpoint_name=sk_endpoint_name)
print("Success!")

### Register in ECS

After the image is built and pushed to Amazon ECR, Amazon SageMaker invokes the training service for your algorithm by running the Docker command we introduced earlier. You can use the Amazon SageMaker console to tell SageMaker the registry path where your own training image is stored in Amazon ECR.

In [32]:
bucket

'sagemaker-us-east-1-678287862391'

In [43]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(bucket)

m_pths = []
for object_summary in my_bucket.objects.all():
    if object_summary.key.startswith(('randfor','tree')):
        full_path = f"s3://{bucket}/{object_summary.key}"
        m_pths.append(full_path)

s3://sagemaker-us-east-1-678287862391/randfor-model-2022-04-16-19-26-51-954/source/sourcedir.tar.gz
s3://sagemaker-us-east-1-678287862391/tree-model-2022-04-16-19-16-07-937/source/sourcedir.tar.gz


In [46]:
model_url = m_pths[0]
model_url2 = m_pths[1]

## Create Model definition

In [ ]:
from sagemaker.image_uris import retrieve

model_name = f"tree-pred-{datetime.now():%Y-%m-%d-%H-%M-%S}"
model_name2 = f"random-for-pred2-{datetime.now():%Y-%m-%d-%H-%M-%S}"
# image_uri = retrieve("xgboost", boto3.Session().region_name, "0.90-1")
# image_uri2 = retrieve("xgboost", boto3.Session().region_name, "0.90-2")

image_uri = get_model()
image_uri2 = get_model()

sm_session.create_model(
    name=model_name, role=role, container_defs={"Image": image_uri, "ModelDataUrl": model_url}
)

sm_session.create_model(
    name=model_name2, role=role, container_defs={"Image": image_uri2, "ModelDataUrl": model_url2}
)

## Create Variants

We set an initial_weight of “1” for both variants: this means 50% of our requests go to Variant1, and the remaining 50% of all requests to Variant2. (The sum of weights across both variants is 2 and each variant has weight assignment of 1. This implies each variant receives 1/2, or 50%, of the total traffic.)

In [ ]:
from sagemaker.session import production_variant

variant1 = production_variant(
    model_name=model_name,
    instance_type="ml.c5.4xlarge",
    initial_instance_count=1,
    variant_name="Variant1",
    initial_weight=1,
)
variant2 = production_variant(
    model_name=model_name2,
    instance_type="ml.c5.4xlarge",
    initial_instance_count=1,
    variant_name="Variant2",
    initial_weight=1,
)

(variant1, variant2)

### Deploy

In [ ]:
endpoint_name = f"DEMO-xgb-churn-pred-{datetime.now():%Y-%m-%d-%H-%M-%S}"
print(f"EndpointName={endpoint_name}")

sm_session.endpoint_from_production_variants(
    name=endpoint_name, production_variants=[variant1, variant2]
)

### Invoke the deployed models

In [ ]:
import itertools
import pandas as pd

a = [50 * i for i in range(3)]
b = [40 + i for i in range(10)]
indices = [i + j for i, j in itertools.product(a, b)]

test_data = df_iris.iloc[indices[:-1]]
test_X = test_data.iloc[:, 1:]
test_y = test_data.iloc[:, 0]

In [ ]:
for index,row in test_X.iterrows():
    print(".", end="", flush=True)
    ayload = row.rstrip("\n")
    sm_runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType="text/csv", Body=payload)
    time.sleep(0.5)
    
print("Done!")

### Invoke a specific variant 1

In [ ]:
import numpy as np

predictions = ""

print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")
with open("./data/val.csv", "r") as f:
    for row in f:
        print(".", end="", flush=True)
        payload = row.rstrip("\n")
        response = sm_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="text/csv",
            Body=payload,
            TargetVariant=variant1["VariantName"],
        )
        predictions = ",".join([predictions, response["Body"].read().decode("utf-8")])
        time.sleep(0.5)

# Convert our predictions to a numpy array
pred_np = np.fromstring(predictions[1:], sep=",")


In [54]:
time.sleep(20)  # let metrics catch up
plot_endpoint_metrics()

### Evaluate the Variant

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
from sklearn.metrics import roc_auc_score

df_with_labels = pd.read_csv("data/val.csv")
test_labels = df_with_labels.iloc[:, 0]
labels = test_labels.to_numpy()

# Calculate accuracy
accuracy = sum(preds == labels) / len(labels)
print(f"Accuracy: {accuracy}")

# Calculate precision
precision = sum(preds[preds == 1] == labels[preds == 1]) / len(preds[preds == 1])
print(f"Precision: {precision}")

# Calculate recall
recall = sum(preds[preds == 1] == labels[preds == 1]) / len(labels[labels == 1])
print(f"Recall: {recall}")

# Calculate F1 score
f1_score = 2 * (precision * recall) / (precision + recall)
print(f"F1 Score: {f1_score}")

# Calculate AUC
auc = round(roc_auc_score(labels, preds), 4)
print("AUC is " + repr(auc))

fpr, tpr, _ = metrics.roc_curve(labels, preds)

plt.title("ROC Curve")
plt.plot(fpr, tpr, "b", label="AUC = %0.2f" % auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

### collect data for Variant2

In [ ]:
predictions2 = ""
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")
with open("./data/val.csv", "r") as f:
    for row in f:
        print(".", end="", flush=True)
        payload = row.rstrip("\n")
        response = sm_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="text/csv",
            Body=payload,
            TargetVariant=variant2["VariantName"],
        )
        predictions2 = ",".join([predictions2, response["Body"].read().decode("utf-8")])
        time.sleep(0.5)

# Convert to numpy array
pred_np2 = np.fromstring(predictions2[1:], sep=",")

# Convert to binary predictions
thresh = 0.5
preds2 = np.where(pred_np2 > threshold, 1, 0)

print("Done!")

In [ ]:
time.sleep(60)  # give metrics time to catch up
plot_endpoint_metrics()

### evaluate variant 2

In [ ]:
# Calculate accuracy
accuracy2 = sum(preds2 == labels) / len(labels)
print(f"Accuracy: {accuracy2}")

# Calculate precision
precision2 = sum(preds2[preds2 == 1] == labels[preds2 == 1]) / len(preds2[preds2 == 1])
print(f"Precision: {precision2}")

# Calculate recall
recall2 = sum(preds2[preds2 == 1] == labels[preds2 == 1]) / len(labels[labels == 1])
print(f"Recall: {recall2}")

# Calculate F1 score
f1_score2 = 2 * (precision2 * recall2) / (precision2 + recall2)
print(f"F1 Score: {f1_score2}")

auc2 = round(roc_auc_score(labels, preds2), 4)
print("AUC is " + repr(auc2))

fpr2, tpr2, _ = metrics.roc_curve(labels, preds2)

plt.title("ROC Curve")
plt.plot(fpr2, tpr2, "b", label="AUC = %0.2f" % auc2)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

### Dialing up our chosen variant in production

we have determined Variant2 to be better as compared to Variant1, we will shift more traffic to it.

A simpler approach is to update the weights assigned to each variant using UpdateEndpointWeightsAndCapacities. This changes the traffic distribution to your production variants without requiring updates to your endpoint

In [56]:
## Recall our variant weights are as follows:
{
    variant["VariantName"]: variant["CurrentWeight"]
    for variant in sm.describe_endpoint(EndpointName=endpoint_name)["ProductionVariants"]
}

In [ ]:
def invoke_endpoint_for_two_minutes():
    with open("./data/val.csv", "r") as f:
        for row in f:
            print(".", end="", flush=True)
            payload = row.rstrip("\n")
            response = sm_runtime.invoke_endpoint(
                EndpointName=endpoint_name, ContentType="text/csv", Body=payload
            )
            response["Body"].read()
            time.sleep(1)

We invoke our endpoint for a bit, to show the even split in invocations:

In [ ]:
invocation_start_time = datetime.now()
invoke_endpoint_for_two_minutes()
time.sleep(20)  # give metrics time to catch up
plot_endpoint_metrics(invocation_start_time)

Now let us shift 75% of the traffic to Variant2 by assigning new weights to each variant using UpdateEndpointWeightsAndCapacities. Amazon SageMaker will now send 75% of the inference requests to Variant2 and remaining 25% of requests to Variant1.

In [ ]:
sm.update_endpoint_weights_and_capacities(
    EndpointName=endpoint_name,
    DesiredWeightsAndCapacities=[
        {"DesiredWeight": 25, "VariantName": variant1["VariantName"]},
        {"DesiredWeight": 75, "VariantName": variant2["VariantName"]},
    ],
)

In [ ]:
print("Waiting for update to complete")
while True:
    status = sm.describe_endpoint(EndpointName=endpoint_name)["EndpointStatus"]
    if status in ["InService", "Failed"]:
        print("Done")
        break
    print(".", end="", flush=True)
    time.sleep(1)

{
    variant["VariantName"]: variant["CurrentWeight"]
    for variant in sm.describe_endpoint(EndpointName=endpoint_name)["ProductionVariants"]
}

Now let's check how that has impacted invocation metrics:

In [ ]:
invoke_endpoint_for_two_minutes()
time.sleep(20)  # give metrics time to catch up
plot_endpoint_metrics(invocation_start_time)